# Juliaで非線型方程式の解の精度保証付き数値計算

非線形方程式

$$
f(x) = 0,\quad f:\mathbb{R}^n \rightarrow \mathbb{R}^n
$$

の解を精度保証することを考える。


$x = x - Df(x)^{-1}f(x)$

`X = verifynlss(function,xinit(point))`を作ってほしい。


今回は、区間解析の標準的な手法となっているR. Krawczykによる解の検証手法を紹介する。

## Krawczyk法

Krawczykの主張は、
    $X\in \mathbb{R}^n$を区間ベクトル（候補者集合）、$c = \mathrm{mid}(X)$、$R\simeq Df(c)^{-1}=J(c)^{-1}$、$E$ を単位行列とし、

\begin{equation}
K(X) = c-Rf(c)+(E-RDf(X))(X-c)
\end{equation}

としたとき、$K(X)\subset \mathrm{int}(X)$ ならば $X$ に $f(x)=0$ の解が唯一存在する。
というものである。（$\mathrm{int}(X)$：$X$ の内部）

この主張を成り立たせるためには、$g(x)=x-Rf(x)$ に対して、平均値形式と縮小写像の原理を適用する必要がある。

### Krawczyk写像

$X\in \mathbb{IR}^n$ を区間ベクトルとし、写像 $f$ に対する簡易ニュートン写像 $g:\mathbb{R}^n\rightarrow \mathbb{R}^n$ を次で定義する。

$$
g(x)=x-Rf(x)
$$

$R\in \mathbb{R}^{n\times n}$ は、$n$ 次元正方行列として、$c\in X$ におけるヤコビ行列 $J(c)$ の逆近似行列 $(R\simeq f'(c)^{-1})$ とする。

このとき、$R$ が正則であるならば、

$$
f(x)=0\Longleftrightarrow g(x)=x
$$

が成り立つ。

そして、$c$ が、区間ベクトル $X$ から $X$ への縮小写像となれば、縮小写像の原理から、$f(x)=0$ の真の解 $x^*$ が $X$ 内に一意存在することが示せる。しかし、写像 $g$ の区間 $X$ における区間拡張 $s_{[]}(X)$ を考えると、常に $s_{[]}(X)=X-Rf_{[]}(X)\not\subset X$ となり、縮小写像の原理を示すことができない。そこで、区間演算における区間の増大を抑制するための基本手法である平均値形式を導入する。

$$
s(X):=\{s(x)\mid x\in X\} (\mbox{写像 $s$ の値域})
$$

$$
s_{[]}(X)\supset s(X)
$$

## 縮小写像の原理

$X\subseteq \mathbb{R}^n$として、写像$T:X\rightarrow X$を考える。このとき、全ての$x,y\in X$において、距離空間$d(x,y)$が

$$
d(T(x),T(y))\le \alpha d(x,y)
$$

が成り立つとき、このような$T$を縮小写像という。（ただし、$\alpha\in [0,1)$ ）

このとき、$g(x) = x$を満たす点$x^*$が$X$に唯一存在する。（不動点という）

## 平均値形式

写像$f: D \rightarrow \mathbb{R}^{n}$が区間$X\subset D$において、1回連続微分可能とする。
このとき、$x, \tilde{x} \in X$に対して、

$$
f(x) \in f(\tilde{x})+D f(X)(x-\tilde{x})
$$

が成立する。
（$Df(x)$は、写像$f$のヤコビ行列$J(x)$の区間$X$における区間拡張）

## 自動微分を利用したヤコビ行列の計算

Kwawczyk法を使う際には、区間拡張 $f_{[]}'(X)$ を計算する必要がある。計算の方法として、最も標準的な実装方法は、自動微分を利用することである。

Juliaで自動微分を適用する場合には、ForwardDiffパッケージを利用する。使用例は以下の通りである。以下では $f(x,y) = x^2+y^2-1$, $g(x,y) = x^3 + y^4$ として

$$
    h(x,y) = \left[\begin{array}{c}f(x,y)\\ g(x,y)\end{array}\right]
$$

のヤコビ行列

$$
J(x) = \left[\begin{array}{cc}f_x(x,y) & f_y(x,y)\\ g_x(x,y) & g_y(x,y)\end{array}\right]= \left[\begin{array}{cc} 2x & 2y\\ 3x^2 & 4y^3\end{array}\right]
$$

の区間 $(x,y) = ([0.8,0.9], [-1,-0.5])$ における値を求める。

In [1]:
using LinearAlgebra,IntervalArithmetic, ForwardDiff
X = [(0.8..0.9),(-1..(-0.5))]
# mid(X)
# X = rand(2)
f(x, y) = x^2 + y^2 - 1
g(x, y) = x^3 + y^4
h( (x, y) ) = [f(x, y); g(x, y)]
# ForwardDiff.jacobian(g, X::IntervalBox) = ForwardDiff.jacobian(g, X.v)
J = ForwardDiff.jacobian(h,X)

2×2 Array{Interval{Float64},2}:
 [1.59999, 1.80001]  [-2, -1]
 [1.91999, 2.43001]   [-4, -0.5]

$h(x,y)$ の区間　$X = ([0.8,0.9], [-1,-0.5])$ における区間拡張 $h_{[]}(X)$ は

In [2]:
h(X)

2-element Array{Interval{Float64},1}:
 [-0.110001, 0.810001]
  [0.574499, 1.72901]

## Krawczyk法の実装

ここから、JuliaにおけるKrawczyk法の実装を行う。実装例は以下の通りである。

In [3]:
using LinearAlgebra,IntervalArithmetic, ForwardDiff
#区間Xを定義
X = [(0.6..0.8),(0.6..0.8)]

#計算対象となる方程式を定義
f(x, y) = x^2 + y^2 - 1
g(x, y) = x - y
F( (x, y) ) = [f(x, y); g(x, y)]
# ForwardDiff.jacobian(g, X::IntervalBox) = ForwardDiff.jacobian(g, X.v)
#J=f'(I)

#区間Xにおけるヤコビ行列を計算
iDF = ForwardDiff.jacobian(F,X) # 区間演算必要

2×2 Array{Interval{Float64},2}:
     [1.19999, 1.60001]        [1.19999, 1.60001]
 [1, 1]                  [-1, -1]

In [4]:
#mid(X)を定義
c = mid.(X)
ic = map(Interval,c)
#Rを計算するためのf'(c)を求める
DF = ForwardDiff.jacobian(F,c) # 区間演算なしでOK

2×2 Array{Float64,2}:
 1.4   1.4
 1.0  -1.0

In [5]:
#区間Xにおけるヤコビ行列Jの逆行列Rを定義する
R = inv(DF) # 区間演算なし

2×2 Array{Float64,2}:
 0.357143   0.5
 0.357143  -0.5

In [6]:
#単位行列を生成
# Matrix{Float64}(I,2,2)

M = Matrix{Float64}(I,2,2) - R*iDF　# 区間演算必要

2×2 Array{Interval{Float64},2}:
 [-0.0714286, 0.0714286]  [-0.0714286, 0.0714286]
 [-0.0714286, 0.0714286]  [-0.0714286, 0.0714286]

In [7]:
R*F(ic)

2-element Array{Interval{Float64},1}:
 [-0.00714286, -0.00714285]
 [-0.00714286, -0.00714285]

In [8]:
#Krawczyk写像を計算
K = c - R*F(ic) + M*(X - c)　# 区間演算必要

2-element Array{Interval{Float64},1}:
 [0.692857, 0.721429]
 [0.692857, 0.721429]

In [9]:
#収束判定
K .⊂ X

2-element BitArray{1}:
 1
 1

In [10]:
function krawczyk(F,X)
    iDF = ForwardDiff.jacobian(F,X);
    c = mid.(X); ic = map(Interval,c);
    DF = ForwardDiff.jacobian(F,c);
    R = inv(DF);
    M = Matrix{Float64}(I,2,2) - R*iDF;
    return c - R*F(ic) + M*(X - c)
end

krawczyk (generic function with 1 method)

In [11]:
K = krawczyk(F,X);
tol = 5e-10;
while maximum(radius.(K)) >= tol
    K = krawczyk(F,K)
end
@show K
radius.(K)

K = Interval{Float64}[[0.707106, 0.707107], [0.707106, 0.707107]]


2-element Array{Float64,1}:
 1.9761969838327786e-14
 1.9761969838327786e-14

In [12]:
X = [(-1..0.8),(0.6..0.8)]
K = krawczyk(F,X);
K .⊂ X

2-element BitArray{1}:
 0
 0

上記のように、候補区間Xの選び方次第では、$K(X)\subset int(X)$が成立しない場合もある。その時は、柏木の方法によって、候補区間$X$を変更する必要がある。以下に、その手順を示す。

$c\in \mathbb{F}^m$を非線形方程式の数値計算で得られた近似解とする。
ここで、$r=|Rf(c)|\in \mathbb{F}^m$をベクトルとして考え、候補区間$X$を

$$
X=\left[\begin{array}{c}
{\left[c_{1}-u_{1}, c_{1}+u_{1}\right]} \\
{\left[c_{2}-u_{2}, c_{2}+u_{2}\right]} \\
\vdots \\
{\left[c_{m}-u_{m}, c_{m}+u_{m}\right]}
\end{array}\right], u_{i}=r_{i}+\frac{1}{n} \Sigma_{k} r_{k}
$$

とする。そうすることで、$K(X)\subset int(X)$がより成立するようになる。以下に実装方法を示す。

In [13]:
#rを計算
r = abs.(R*F(c))

2-element Array{Float64,1}:
 0.007142857142857189
 0.007142857142857189

In [14]:
#uを計算
u = r .+ (sum(r)/length(r))

2-element Array{Float64,1}:
 0.014285714285714377
 0.014285714285714377

In [15]:
#候補区間Xを新たに定める
X = c .± r

2-element Array{Interval{Float64},1}:
 [0.692857, 0.707143]
 [0.692857, 0.707143]

**matlabだと、midrad()で計算できる？**

### 参考文献

1. 大石進一編著, 精度保証付き数値計算の基礎, コロナ社, 2018.<br>
(精度保証付き数値計算の教科書. 浮動小数点数および区間演算に詳しい. 今回は6章を参考にした)
1. 非線形方程式の解の精度保証 (+ 自動微分)<br>
http://www.kashi.info.waseda.ac.jp/~kashi/lec2020/nac/krawczyk.pdf